In [10]:
# 1. Install & import
!pip install opencv-python-headless matplotlib imageio

import cv2
import imageio
from google.colab import files
import os

# # 2. Upload your files (query2.jpg & video.mp4)
# print("Upload your query image (query.png) and video file (input.mp4):")
# uploaded = files.upload()

# # 3. Verify uploads
# if 'query.png' not in uploaded or 'input.mp4' not in uploaded:
#     raise FileNotFoundError("Please upload both 'query2.jpg' and 'video.mp4'.")

# 4. Load the query image in grayscale and set up ORB/BFMatcher
query = cv2.imread('query.png', cv2.IMREAD_GRAYSCALE)
orb = cv2.ORB_create()
kp_query, des_query = orb.detectAndCompute(query, None)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# 5. Open the video
cap = cv2.VideoCapture('input.mp4')
if not cap.isOpened():
    raise FileNotFoundError("Couldn't open video.mp4")

# 6. Process frames and collect matched images
frames = []
max_frames = 10
for i in range(max_frames):
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = orb.detectAndCompute(gray, None)
    if des_frame is not None:
        matches = bf.match(des_query, des_frame)
        good = sorted(matches, key=lambda m: m.distance)[:20]
        matched = cv2.drawMatches(
            query, kp_query, frame, kp_frame, good, None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        # Add the frame number as text on the image
        frame_text = f"Frame {i+1}"
        cv2.putText(matched, frame_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Convert BGR → RGB for GIF
        frames.append(cv2.cvtColor(matched, cv2.COLOR_BGR2RGB))

cap.release()

# 7. Write out GIF
gif_path = '/content/matched_with_frame_numbers.gif'
imageio.mimsave(gif_path, frames, fps=2)  # 2 frames per second

# 8. Download
print("Saving animated GIF with frame numbers…")
files.download(gif_path)


Saving animated GIF with frame numbers…


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>